# WALLABY Database Access Notebook

<span style="font-weight: bold; color: #FF0000;">⚠ Make sure the Jupyter Notebook server is loaded with the wallaby/python-3.9.1 module!</span>

<span style="font-weight: bold; color: #FF0000;">⚠ If the Jupyter Notebook server is not loaded with the wallaby/python-3.9.1 delete the interactive session and start a new one with the correct module.</span>

## Connect to Database

Connect to the WALLABY database by importing the `wallaby_data_access` module and calling the `wallaby.connect()` function. This uses the read-only `wallaby_user` database credentials to access the database.

In [ ]:
import wallaby_data_access as wallaby
wallaby.connect()

---

# Source products

## Retrieve catalog

To get the source table as an Astropy table object use the `wallaby.get_catalog()` function. Catalogs are retrieved by tag, which are define different collections of sources. You can view the available tags with `wallaby.print_tags()`.

In [ ]:
wallaby.print_tags()

As an example, let us retrieve all sources from phase 2 pilot observations released as part of the NGC 5044 DR1 release by supplying the `NGC 5044 DR1` tag to the `wallaby.get_catalog()` function:

In [ ]:
# Set tag name

tag_name = "NGC 5044 DR1"

In [ ]:
# Retrieve catalog as Astropy table

from astropy.table import Table

table = wallaby.get_catalog(tag_name);

# Sort table by flux (brightest first)
table.sort("f_sum", reverse=True)

# Print table
table.pprint()

The source catalog returned by the function should have been printed above (if not, check for error messages) and is stored in the variable `table`. We can now use basic indexing to access different catalog entries. For example, `table["f_sum"]` will return the entire column of integrated flux measurements, and we can use `table["f_sum"][0]` etc. to extract the individual fluxes for each source. Likewise, `table[0]` will extract the entire first row of the catalog, i.e. a list of all parameters of the first source.

## Calculate Physical Parameters

The next example demonstrates how to retrieve certain parameters from the catalog and use basic arithmetic to convert some of the raw measurements made by SoFiA into physically meaningful parameters such as redshift or HI mass. These can be directly appended to the catalog as additional columns using `table["parameter_name"] = <expression>`.

In [ ]:
import numpy as np
import scipy.constants as const
from astropy.cosmology import FlatLambdaCDM

# Set up cosmology
f_rest = 1.42040575e+9;  # HI rest frequency in Hz
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

# Calculate redshift
table["redshift"] = f_rest / table["freq"] - 1.0

# Calculate luminosity distance in Mpc and HI mass in solar masses
table["dl"] = cosmo.luminosity_distance(table["redshift"]).value
table["log_mhi"] = np.log10(49.7 * table["dl"] * table["dl"] * table["f_sum"])

# Calculate source rest frame velocity width in km/s
table["dv"] = const.c * (1.0 + table["redshift"]) * table["w20"] / f_rest / 1000.0

# Show our new parameters
table["name", "id", "redshift", "dl", "log_mhi", "dv"].pprint(max_width=-1)

## Create a Plot

Once we’ve done our analysis, we can the create plots of any of the parameters in our table. In this example, let us plot the logarithmic HI mass against redshift and additionally colour the data points by source rest frame velocity width. If desired, the resulting plot can be exported as a PDF file and then downloaded to your local computer, e.g. to use in a presentation or publication.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16

plt.scatter(table["redshift"], table["log_mhi"], s=16, c=table["dv"], cmap="jet")
plt.xlabel(r"$z$")
plt.ylabel(r"$\log_{10}(M_{\rm HI} / M_{\odot})$")
cbar = plt.colorbar()
cbar.set_label(r"$\Delta v \; (\mathrm{km \, s}^{-1})$")
plt.xlim(0.0, 0.1)
plt.ylim(7.0, 11.0)
plt.grid(True)

# Uncomment the following line to make a PDF copy in the notebook folder for download
#plt.savefig("my_plot.pdf", format="pdf", bbox_inches="tight", pad_inches=0.05)

plt.show()

## Filtering the catalog

Once we have the catalog loaded into an Astropy table object, we can easily make selections to suit our scientific needs. The following examples illustrate how the catalog can be filtered by certain criteria such as parameter ranges or the presence of comments and tags.

**Example 1: Filter sources by parameter range**

In [ ]:
# Select all sources within a certain RA and Dec range

mask = (table["ra"] > 202.0) & (table["ra"] < 203.0) & (table["dec"] > -22.5) & (table["dec"] < -21.5)
table[mask].pprint()

**Example 2: Filter sources tagged as components of a galaxy**

In [ ]:
# Select all sources that have the "Component" tag set

mask = ["Component" in tags for tags in table["tags"]]
table[mask]["name", "id", "tags"].pprint_all()

**Example 3: Filter sources that have comments attached**

In [ ]:
# Select all sources with at least one comment

mask = [len(comments) > 0 for comments in table["comments"]]
table[mask]["name", "id", "comments"].pprint_all()

## Get the overview plot for a specific source

It is also possible to display an overview plot of a specific source (as identified by its catalog ID) by calling the `wallaby.overview_plot()` function. That function will display four panels showing the moment 0 and 1 maps, a DSS image with HI contours and the integrated spectrum of the source. **Note that it may take up to half a minute before the plot is displayed, as Astropy must download the DSS image from Skyview first.** If the Skyview download fails, which happens occasionally, just try again a few hours later.

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

plt = wallaby.overview_plot(id=4713, size=(10,8))
plt.show()

---

# Kinematic models

Kinematic models are now available through the WALLABY database. Only a subset of the sources have kinematic models. They can be accessed in a similar way to source products. First, we list the tags with `wallaby.get_kinematic_model_tags()`. Then, we use the tag to retrieve the catalog as an Astropy table with `wallaby.get_kinematic_model()`.

The kinematic model tags use the convention `TR` for "team release" and have the "Kin" keyword, whereas the source finding models use `DR` for "data release". To retrieve the kinematic models associated with NGC 5044 DR2, you would select the kinematic model tag "NGC 5044 Kin TR2".

In [ ]:
# List kinematic model tags

wallaby.get_kinematic_model_tags()

In [ ]:
# Select a tag

kin_tag = 'NGC 5044 Kin TR1'

In [ ]:
kin_table = wallaby.get_kinematic_model(team_release_kin=kin_tag)

In [ ]:
# Show first two rows of table

kin_table[0:2]

We can then use the kinematic model table in the same way as the source finding table. Below is an example figure created from the kinematic model catalog.

In [ ]:
%matplotlib inline

#To make the plot look a bit nicer, we'll set the figure size and font size
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16

for row in kin_table:
    r = np.array(row['rad'])
    v = np.array(row['vrot_model'])
    plt.plot(r, v, ls='-', color='k', marker='')
    
plt.title(kin_tag)
plt.xlabel(r"$R$ ('') ")
plt.ylabel(r"$V_{rot}$ (km/s)")
plt.xlim(0.0, 200)
plt.ylim(0, 350)
plt.grid(True)
plt.show()